# Simple Impementation of E(n) Equivariant Graph Neural Networks

Original paper https://arxiv.org/pdf/2102.09844.pdf by Victor Garcia Satorras, Emiel Hoogeboom, Max Welling

In [1]:
# !pip install torch --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu116
# Установка PyTorch с поддержкой CUDA 11.6

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu116


In [2]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.cuda.is_available()

True

# Load QM9 Dataset

In [14]:
!git clone https://github.com/senya-ashukha/simple-equivariant-gnn.git
%cd simple-equivariant-gnn

C:\Users\kulev\simple-equivariant-gnn\simple-equivariant-gnn


Cloning into 'simple-equivariant-gnn'...


In [31]:
# QM9 is a dataset for Molecular Property Predictions http://quantum-machine.org/datasets/
# We will predict Highest occupied molecular orbital energy 
# https://en.wikipedia.org/wiki/HOMO_and_LUMO
# We use data loaders from the official repo

from qm9.data_utils import get_data, BatchGraph
train_loader, val_loader, test_loader, charge_scale = get_data(num_workers=0)

# Graph Representation

In [32]:
batch = BatchGraph(iter(train_loader).next(), False, charge_scale)
batch

AttributeError: ignored

# Define Equivariant Graph Convs  & GNN

In [33]:
def index_sum(agg_size, source, idx, cuda):
    """
        source is N x hid_dim [float]
        idx    is N           [int]
        
        Sums the rows source[.] with the same idx[.];
    """
    tmp = torch.zeros((agg_size, source.shape[1]))
    tmp = tmp.cuda() if cuda else tmp
    res = torch.index_add(tmp, 0, idx, source)
    return res

In [34]:
class ConvEGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        self.cuda = cuda
        
        self.f_e = nn.Sequential(nn.Linear(2 * in_dim + 1, hid_dim), nn.ReLU(), nn.Linear(hid_dim, hid_dim), nn.ReLU())
        
        self.f_inf = nn.Sequential(nn.Linear(hid_dim, 1), nn.Sigmoid()) 
        
        self.f_h = nn.Sequential(nn.Linear(hid_dim + in_dim, hid_dim), nn.SiLU(), nn.Linear(hid_dim, hid_dim))
    
    def forward(self, b):
        # compute distances for all edges
        e_st, e_end = b.edges[:,0], b.edges[:,1]
        dists = torch.norm(abs(b.x[e_st] - b.x[e_end]), dim=1).reshape(-1, 1)
        
        # compute messages
        tmp = torch.hstack([b.h[e_st], b.h[e_end], dists])
        m_ij = self.f_e(tmp)
        
        # predict edges
        e_ij = self.f_inf(m_ij)
        
        # average e_ij-weighted messages  
        # m_i is num_nodes x hid_dim
        m_i = index_sum(b.h.shape[0], e_ij*m_ij, b.edges[:,0], self.cuda)
        
        # update hidden representations
        tmp = torch.hstack([m_i, b.h])
        b.h += self.f_h(tmp)

        return b

In [35]:
class NetEGNN(nn.Module):
    def __init__(self, in_dim=15, hid_dim=128, out_dim=1, n_layers=7, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        
        self.emb = nn.Linear(in_dim, hid_dim) 

        # Make gnn of n_layers
        self.gnn = nn.Sequential(*[ConvEGNN(hid_dim, hid_dim, cuda=cuda) for _ in range(n_layers)])
      
        self.pre_mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
        
        self.post_mlp = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, out_dim))

        if cuda: self.cuda()
        self.cuda = cuda
    
    def forward(self, b):
        b.h = self.emb(b.h)
        
        b = self.gnn(b)
        h_nodes = self.pre_mlp(b.h)
        
        # h_graph is num_graphs x hid_dim
        h_graph = index_sum(b.nG, h_nodes, b.batch, self.cuda) 
        
        out = self.post_mlp(h_graph)
        return out

In [36]:
epochs = 1000
cuda = True

model = NetEGNN(n_layers=7, cuda=cuda)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, verbose=False)

# Training

In [37]:
print('> start training')

tr_ys = train_loader.dataset.data['homo'] 
me, mad = torch.mean(tr_ys), torch.mean(torch.abs(tr_ys - torch.mean(tr_ys)))

if cuda:
    me = me.cuda()
    mad = mad.cuda()

train_loss = []
val_loss = []
test_loss = []

for epoch in range(epochs):
    print('> epoch %s:' % str(epoch).zfill(3), end=' ', flush=True) 
    start = time.time()

    batch_train_loss = []
    batch_val_loss = []
    batch_test_loss = []

    model.train()
    for batch in train_loader:
        batch = BatchGraph(batch, cuda, charge_scale)
        
        out = model(batch).reshape(-1)
        
        loss =  F.l1_loss(out, (batch.y - me) / mad)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            loss =  F.l1_loss(out*mad+me, batch.y)

        batch_train_loss += [float(loss.data.cpu().numpy())]  
        
    train_loss += [np.mean(batch_train_loss)/0.001]
    
    print('train %.3f' % train_loss[-1], end=' ', flush=True)
    
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_val_loss += [np.mean(loss)]
            
        val_loss += [np.mean(batch_val_loss)/0.001]
        
        print('val %.3f' % val_loss[-1], end=' ', flush=True)
        
        for batch in test_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_test_loss += [np.mean(loss)]

        test_loss += [np.mean(batch_test_loss)/0.001]
        
    end = time.time()

    print('test %.3f (%.1f sec)' % (test_loss[-1], end-start), flush=True)
    lr_scheduler.step()

> start training
> epoch 000: train 347.839 val 292.940 test 288.890 (115.6 sec)
> epoch 001: train 236.499 val 215.980 test 215.799 (108.8 sec)
> epoch 002: train 189.462 val 177.226 test 177.115 (111.3 sec)
> epoch 003: train 166.638 val 142.420 test 140.712 (109.4 sec)
> epoch 004: train 151.305 val 140.691 test 139.867 (110.3 sec)
> epoch 005: train 140.757 val 137.295 test 137.778 (111.0 sec)
> epoch 006: train 133.721 val 129.073 test 129.840 (110.2 sec)
> epoch 007: train 125.512 val 114.223 test 113.974 (110.7 sec)
> epoch 008: train 119.024 val 113.256 test 111.740 (108.9 sec)
> epoch 009: train 114.211 val 105.659 test 105.743 (108.5 sec)
> epoch 010: train 111.649 val 100.730 test 101.223 (107.4 sec)
> epoch 011: train 106.646 val 99.395 test 99.002 (107.3 sec)
> epoch 012: train 104.310 val 97.588 test 97.101 (107.3 sec)
> epoch 013: train 101.760 val 95.613 test 94.903 (107.2 sec)
> epoch 014: train 99.546 val 93.259 test 93.540 (107.4 sec)
> epoch 015: train 97.121 val 90

KeyboardInterrupt: ignored

In [ ]:
# > epoch 280: train 38.413 val 42.188 test 42.738 (107.6 sec)
# Дальше модель начала переобучаться

In [ ]:
# This is good results 
# Lower is better
# val 30.157 test 30.886